In [ ]:
import torch
import polyak


x = torch.randn(2, requires_grad=True, dtype=torch.double)
def loss_function():
    return sum(torch.abs(x))

def closure():
    optimizer.zero_grad()
    loss = loss_function()
    loss.backward()
    return loss

param = [x]
optimizer = polyak.Polyak(param)
# A short loop that applies the Polyak subgradient method to the loss function
for i in range(100):
    loss = optimizer.step(closure)
    print("Iteration: {}, Loss: {}".format(i, loss))
    print("x: {}".format(x.data))
